In [1]:
from sklearn.cluster import KMeans
from sklearn.metrics import f1_score
from sklearn.metrics import homogeneity_completeness_v_measure
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.io as sio
import numpy as np
import cv2
import glob
from PIL import Image
import regex
from sklearn.cluster import SpectralClustering
import operator
import pandas as pd

# Reading images, extracting groundTruth and boundries 

In [2]:
def read_images():
    image_samples=[]
    boundries=[]
    segmentation_Gt=[]
    images_list=[]
    path= 'BSR/BSDS500/data/images/train/'
    path_mat= 'BSR/BSDS500/data/groundTruth/train/'
    for filename in glob.glob(path+"*.jpg"): 
        im=Image.open(filename) #read file name 
        img = np.array(im) # convert pil to numpy array
        images_list.append(img) 
        image_samples.append(img.reshape(img.shape[0]*img.shape[1], 3)) # reshape to samples 
        image_number=regex.split(r'.jpg',regex.split(r'/', filename)[5])[0] # get the number of the image
        temp= sio.loadmat(path_mat+image_number) # read boundries, groundtruth
        boundries.append(temp['groundTruth'][0][0]['Boundaries'][0][0]) #extract boundries 
        segmentation_Gt.append(temp['groundTruth'][0][0]['Segmentation'][0][0]) #extract ground truth
    return images_list, image_samples, boundries, segmentation_Gt

# utilities

In [3]:
def visualize(img,segmentation_Gt,boundries ,segmentation_pred,i):
    m=plt.figure()
    plt.imshow(img)
    
    s=plt.figure()
    plt.imshow(segmentation_Gt)
    
    
    b=plt.figure()
    plt.imshow(boundries)
   
    d=plt.figure()
    plt.imshow(segmentation_pred)

    

In [4]:
def evaluate(y_pred, y_true):
    #if (meaure=="F_measure"):
    F_measure_score= f1_score(y_true, y_pred, average=None)
    #else if(measure== "conditional_entropy"):
    conditional_entropy_score= homogeneity_completeness_v_measure(y_true, y_pred)
    #print("F-measure=", F_measure_score, "\n conditional entropy score=",conditional_entropy_score )
    return F_measure_score, conditional_entropy_score

In [5]:
k=[3,5,7,9,11]
def K_means(X):
    sdict=dict()
    for i in k:
        kmeans = KMeans(n_clusters=i, random_state=0).fit(X)
        sdict[i]=kmeans.labels_
   
    return sdict

In [6]:
def NN_cut():
    SpectralClustering(n_clusters=5, affinity="nearest_neighbors", n_neighbors=5).fit(images_list[0])     
    return 

# Training with K- clusters and get best K

In [7]:
images_list,image_samples, boundries, segmentation_Gt= read_images() #read images

In [ ]:
#loop through images and visualize, cluster with kmeans and normalizd_cut 
#and evaluate using F_measure and condtional entropy
PicsNum=5
f_measures=np.zeros((PicsNum,len(k)))
entropy_score=np.zeros((PicsNum,len(k)))
segmentation_pred=[]
best_k=[]
listOfDicts=[]
for i in range(PicsNum):
    counter=0
    km=K_means(image_samples[i])
    listOfDicts.append(km)    
    for j in range(3,12,2):
        y_pred=km[j]
        y_true=segmentation_Gt[i].reshape(segmentation_Gt[i].shape[0]*segmentation_Gt[i].shape[1])
        F_measure_score, conditional_entropy_score= evaluate(y_pred, y_true)
        f_measures[i,counter]=np.sum(F_measure_score)
        entropy_score[i,counter]=np.sum(conditional_entropy_score)
        counter+=1
    t=np.argmax(f_measures[i])
    best_k.append((t+2)*2-1)
    segmentation_pred.append(np.asarray(km[best_k[i]]).reshape(segmentation_Gt[i].shape[0],segmentation_Gt[i].shape[1]))
    visualize(images_list[i],segmentation_Gt[i],boundries[i],segmentation_pred[i],i)

/home/alaasameh/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/alaasameh/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
pd.DataFrame(data=f_measures, columns=[3,5,7,9,11]) 

# Testing

In [ ]:
KMeans(n_clusters=k, random_state=0).predict(images_list[100])
visualize(images_list[i],segmentation_Gt[i],boundries[i],segmentation_pred[i],i)